In [1]:
'''
手机抓包
'''

import re , urllib , http.cookiejar , base64 , binascii , rsa, json, time
import sqlite3
from log import log

__version__ = '0.0.1'
__author__ = 'Pidong Li (hope-dream@163.com)'
cj = http.cookiejar.LWPCookieJar()
cookie_support = urllib.request.HTTPCookieProcessor(cj)
opener = urllib.request.build_opener(cookie_support , urllib.request.HTTPHandler)
urllib.request.install_opener(opener)



In [2]:
headers = {
"User-Agent":" Dalvik/1.6.0 (Linux; U; Android 4.1.1; MI 2A MIUI/4.12.5)",
"Host":" cy.zhubaotown.com",
"Connection":" Keep-Alive",
"Accept-Encoding":" gzip",
"Content-Type":" application/x-www-form-urlencoded"
}

In [3]:
token = 'f0d4992ab910290d9cc8716f16f61b5430de4141e3321a25fa12bb627421502beb7aac2f'

In [4]:
def get_shop_list(page, size):

    '''
        获得商家列表，以json导出
    '''

    para_shop_list = 'orderKeyword=desc&orderValue=sort_order&page=%d&screenRegionId=0&size=%d&uId=16891&token=%s\
    &bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (page, size, token)
    url_shop_list = 'http://cy.zhubaotown.com/index.php/shop/allShopList/'
    data = para_shop_list.encode('utf-8')

    request = urllib.request.Request(url_shop_list , data, headers)
    response = urllib.request.urlopen(request)
    text = response.read().decode('utf-8')
    out = json.loads(text)
    return out['datas']


In [5]:
# 连接数据库
con = sqlite3.connect('cuiyuan.db')
cu = con.cursor()


In [6]:
# 商家列表 shop_list 写入数据库，不清楚具体的商铺数目，先试10000个
def db_shop_list():
    cu.execute('create table if not exists shop_list(shopId int, shopLogo varchar(255), shopMaster varchar(100), \
    shopName varchar(255), shopSn varchar(255), shopTotalGoods int)')
    con.commit()
    shops = []
    for ii in range(100):
        try:
            print(ii+1)
            shops = get_shop_list(ii+1, 100)
            for shop in shops:
                cu.execute(r"insert into shop_list values(?, ?, ?, ?, ?, ?)", (shop['shopId'], shop['shopLogo'], shop['shopMaster'], shop['shopName'], shop['shopSn'], shop['shopTotalGoods']))
            con.commit()
        except Exception as e:
            print(e)


In [7]:
# 获取 商铺的id和商铺拥有的物品数目
cu.execute('select distinct shopId, shopTotalGoods from shop_list')
shop_ids = cu.fetchall()

In [8]:
# 得到商铺的id
shop_ids_1 = []  
for x in shop_ids:
    shop_ids_1.append(x[0])

In [7]:
# 根据商铺数据，得到物品的id，这是一页的
def shop2goods_id_page(page, shop_id, size, token):
    para_goods_id = 'goodsNew=0&isRecommended=0&page=%d&shopId=%d&size=%s&uId=16891&token=%s\
    &bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (page ,shop_id,  size, token)
    url_goods_id = 'http://cy.zhubaotown.com/index.php/shop/shopGoodsList_new/'
    data = para_goods_id.encode('utf-8')

    request = urllib.request.Request(url_goods_id , data, headers)
    response = urllib.request.urlopen(request)
    text = response.read().decode('utf-8')
    xx = json.loads(text)['datas']
    return(xx)

In [10]:
# 根据商铺数据，得到全部物品的id，
def shop2goods_id(shop_id, shopTotalGoods, token, size = 100):
# 获得商家的物品
    goods = []
    if shopTotalGoods>100 :
        for page in range(math.floor(shopTotalGoods/size)+1):
            xx = shop2goods_id_page(page + 1, shop_id, size, token)
            goods = goods + xx
    elif 0< shopTotalGoods <101:
        xx = shop2goods_id_page(1, shop_id, shopTotalGoods, token)
        goods = goods + xx
    else:
        print('chucuole')
    return goods

In [ ]:
# shop_ids2goods_ids 写入数据库
#新建数据库 shop2goods
def db_shop2goods():
    cu.execute('create table if not exists shop2goods (shop_id int , goods_id int, date varchar(100))')
    con.commit()
    index = 1695
    for shop_id in shop_ids:
        index = index+1
        print(index)
        try:
            goods_list = shop2goods_id(shop_id[0], shop_id[1], token)
            for goods in goods_list:
                cu.execute("insert into shop2goods values(%d, %d, '%s')" %(shop_id[0], goods['goodsId'], goods['add_time']))
        except Exception as e:
            print(e)
        con.commit()

In [8]:
# 获取 goods_info
def get_goods_info(goods_id):
    '''
    通过goods_id获取内容
    '''
    goods_id = str(goods_id)
    para_goods_info = 'goodsId=%s&uId=16891&token=%s&bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (goods_id, token)
    url_goods_info = 'http://cy.zhubaotown.com/index.php/goods/getGoodsInfo/'
    data = para_goods_info.encode('utf-8')
    try:
        request = urllib.request.Request(url_goods_info , data, headers)
        response = urllib.request.urlopen(request)
        text = response.read().decode('utf-8')
        xx = json.loads(text)["goodsInfo"]
    except:
        print(goods_id, 'error')
    
    return xx



In [51]:
# 删除物品数据重新开始
def db_clear_goods_info():
    cu.execute('drop table if exists goods_info')
    cu.execute('drop table if exists goods_img_url')
    # 物品信息写入数据库,21列
    # 创建数据库
    cu.execute('create table if not exists goods_info (goodsId varchar(20), appointmentedStatus varchar(20), \
                goodsInfo varchar(200), shopId varchar(20), title varchar(255),\
                 goodsMaterialId varchar(20), goodsSn varchar(20), goodsTypeId varchar(20),\
                has_cert varchar(20),isAppointmented varchar(20), isConcern varchar(20), isRecommended varchar(20),isimplant varchar(20), orderStatus varchar(20),\
                price real, priceRangeName varchar(255), quantity varchar(20),\
                goodPattern varchar(100), goodPatternId varchar(20), goodsCateId int, goodsCateName varchar(100) )')
    cu.execute('create table if not exists goods_img_url (goods_id int, img_url text)')

    con.commit()

In [9]:

# 得到全部的物品id
cu.execute('select distinct goods_id from shop2goods')
goods_ids = cu.fetchall() 


In [10]:
# 图片url写入数据库

def get_img_url(goods_id, img_arr):
    for img in img_arr:
        cu.execute('insert into goods_img_url values(?, ?)', (goods_id, img['imgUrl']) )



    
    

In [11]:

logger = log('cuiyuan.log', 'goods_info')
def db_goods_info(goods_ids):
    index = 0
    for goods_id in goods_ids:
        index = index +1;
        print(r'已完成 '+ str( index/len(goods_ids)*100) + '%, 请耐心等待', end = '\r')
        try:
            goods_info = get_goods_info(goods_id)
            cu.execute("insert into goods_info values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", \
                       (goods_info['goodsId'], goods_info['appointmentedStatus'], goods_info['goodsInfo'], goods_info['shopId'], \
                        goods_info['title'], goods_info['goodsMaterialId']\
                       , goods_info['goodsSn'], goods_info['goodsTypeId'], goods_info['has_cert'], goods_info['isAppointmented'], goods_info['isConcern']\
                       , goods_info['isRecommended'], goods_info['isimplant'], goods_info['orderStatus'], goods_info['price'], goods_info['priceRangeName'], goods_info['quantity'], goods_info['goodPattern']\
                       , goods_info['goodPatternId'], goods_info['goodsCateId'], goods_info['goodsCateName']))
            get_img_url(goods_id[0], goods_info['goodsImgArr'])
            con.commit()
        except Exception as e:
    #         print(e)
            logger.error('物品信息不完整')
            con.rollback()

In [ ]:
# 查找更新的id

In [68]:

print(len(aa))
cu.execute('select distinct goods_id from goods_img_url') # 46643


46643


In [12]:
# 将数据库返回值的变为list（仅是第一列）
def tuple2list(tuple_list):
    out = []
    for xx in tuple_list:
        out.append(str(xx[0]))
    return out

In [13]:
# 取差集

def update_goods_info(goods_new_ids):
     # 需要另外函数
    goods_new_ids_list = tuple2list(goods_new_ids)
    cu.execute('select distinct goods_id from shop2goods')
    goods_ids = cu.fetchall() 
    goods_ids_list = tuple2list(goods_ids)
    goods_update__ids = list(set(goods_new_ids_list).difference(set(goods_ids_list)))  # goods_new_ids_list - goods_ids_list
    db_goods_info(goods_update__ids)
    
    
    
    
    

In [15]:
cu.execute('select distinct goodsId from goods_info') #46643
goods_new_ids = cu.fetchall()
goods_new_ids_list = tuple2list(goods_new_ids)
cu.execute('select distinct goods_id from shop2goods')
goods_ids = cu.fetchall() 
goods_ids_list = tuple2list(goods_ids)
goods_update__ids = list(set(goods_ids_list).difference(set(goods_new_ids_list)))  # goods_new_ids_list - goods_ids_list


In [ ]:
db_goods_info(goods_update__ids)

In [58]:
cu.close()
con.close()

In [18]:
import csv
cu.execute('select * from goods_info')
goods_info_all = cu.fetchall()
rownames = ['goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' , 'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
csvfile = open('物品信息.csv', 'w')  
writers = csv.writer(csvfile) 
writers.writerow(rownames)
writers.writerows(goods_info_all)
csvfile.close()

In [22]:
cu.execute('select * from shop_list left join goods_info on shop_list.shopId = goods_info.shopId')
shop_goods_all = cu.fetchall()

rownames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods', \
           'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' , 'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
csvfile = open('商家-物品信息utf.csv', 'w', encoding = 'utf-8')  
writers = csv.writer(csvfile) 
writers.writerow(rownames)
writers.writerows(shop_goods_all)
csvfile.close()

In [21]:
cu.execute('select * from goods_img_url')
img_url_all = cu.fetchall()

rownames = ['goodsId', 'goods_img_url']
csvfile = open('物品图片.csv', 'w')  
writers = csv.writer(csvfile) 

writers.writerow(rownames)
writers.writerows(img_url_all)
csvfile.close()

In [ ]:
cu.execute('select * from shop_list left join goods_info on shop_list.shopId = goods_info.shopId')
shop_goods_all = cu.fetchall()

In [43]:


rownames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods', \
           'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' , 'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
csvfile = open('商家-物品信息.csv', 'wb')  
csvfile.write(codecs.BOM_UTF8)
csvfile.close()
csvfile = open('商家-物品信息.csv', 'a', encoding = 'utf-8')  
writers = csv.writer(csvfile) 
writers.writerow(rownames)

for line in shop_goods_all:
    writers.writerow(line)
csvfile.close()

In [28]:
import codecs  


In [45]:
cu.execute('select * from shop_list')
data = cu.fetchall()
rownames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods']
#            'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' , 'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
csvfile = open('商家信息.csv', 'wb')  
csvfile.write(codecs.BOM_UTF8)
csvfile.close()
csvfile = open('商家信息.csv', 'a', encoding = 'utf-8')  
writers = csv.writer(csvfile) 
writers.writerow(rownames)

for line in data:
    writers.writerow(line)
csvfile.close()